In [1]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from argparse import Namespace
from HH4b.postprocessing.PostProcess import load_process_run3_samples


data_folder = "24May24_v12_private_signal"
args = Namespace(
    templates_tag="24June27",
    data_dir="/ceph/cms/store/user/cmantill/bbbb/skimmer/",
    tag="24May24_v12_private_signal",
    years=["2022"],
    training_years=None,
    mass="H2PNetMass",
    bdt_model="24May31_lr_0p02_md_8_AK4Away",
    bdt_config="24May31_lr_0p02_md_8_AK4Away",
    txbb_wps=[0.975, 0.92],
    bdt_wps=[0.98, 0.88, 0.03],
    method="sideband",
    vbf_txbb_wp=0.95,
    vbf_bdt_wp=0.98,
    sig_keys=["hh4b", "vbfhh4b"],
    pt_first=300,
    pt_second=250,
    bdt_roc=False,
    control_plots=False,
    fom_scan=False,
    fom_scan_bin1=True,
    fom_scan_bin2=True,
    fom_scan_vbf=False,
    templates=False,
    legacy=True,
    vbf=True,
    vbf_priority=False,
    weight_ttbar_bdt=1,
    blind=True,
)

In [8]:
bdt_training_keys = ["qcd", "vbfhh4b-k2v0", "hh4b", "ttbar"]
mass_window = np.array([105, 150])
years = ["2022", "2022EE", "2023", "2023BPix"]

ev_dicts = []
for year in years:
    ev_dict, _ = load_process_run3_samples(
        args,
        year=year,
        bdt_training_keys=bdt_training_keys,
        control_plots=False,
        plot_dir="plot_dir",
        mass_window=mass_window,
    )
    ev_dicts.append((year, ev_dict))

"""
python3 PostProcess.py --templates-tag 24June27 --tag 24May24_v12_private_signal --mass H2PNetMass --legacy --bdt-config 24May31_lr_0p02_md_8_AK4Away --bdt-model 24May31_lr_0p02_md_8_AK4Away --txbb-wps 0.975 0.92 --bdt-wps 0.98 0.88 0.03 --vbf-txbb-wp 0.95 --vbf-bdt-wp 0.98 --no-bdt-roc --no-fom-scan --no-fom-scan-bin2 --no-fom-scan-bin1 --data-dir /ceph/cms/store/user/cmantill/bbbb/skimmer/ --method abcd --no-vbf-priority --vbf --no-fom-scan-vbf --pt-second 250 --templates --years 2022 --sig-keys hh4b vbfhh4b
"""
# make array with event nr, signal category
# make mask with signal category cutoffs, apply to array

# save as root

load samples data
Loading JetMET_Run2022D
Loaded JetMET_Run2022D                                   : 249923 entries
Loading JetMET_Run2022C_single
Loaded JetMET_Run2022C_single                            : 44920 entries
Loading JetMET_Run2022C
Loaded JetMET_Run2022C                                   : 386833 entries
JEC shifts  ['']
perform inference
Inference and selection for jshift 
Data cutflow in [105-150] is taken from sideband estimate!
load samples ttbar
Loading TTto2L2Nu
Loaded TTto2L2Nu                                         : 30953 entries
Loading TTto4Q
Loaded TTto4Q                                            : 1008426 entries
Loading TTtoLNu2Q
Loaded TTtoLNu2Q                                         : 381875 entries
scaling and smearing mass for ttbar 2022
JEC shifts  ['', 'JER_up', 'JER_down', 'JES_AbsoluteScale_up', 'JES_AbsoluteScale_down']
perform inference
Inference and selection for jshift 
Inference and selection for jshift JER_up
Inference and selection for jshift

/home/users/dprimosc/HH4b/src/HH4b/utils.py:346: UserWarning: No events for Zto2Q-4Jets_HT-200to400!
  warnings.warn(f"No events for {sample}!", stacklevel=1)
/home/users/dprimosc/HH4b/src/HH4b/utils.py:346: UserWarning: No events for Wto2Q-3Jets_HT-200to400!
  warnings.warn(f"No events for {sample}!", stacklevel=1)


Loaded Zto2Q-4Jets_HT-800                                : 68014 entries
Loading Wto2Q-3Jets_HT-600to800
Loaded Wto2Q-3Jets_HT-600to800                           : 1844 entries
scaling and smearing mass for vjets 2022
JEC shifts  ['', 'JER_up', 'JER_down', 'JES_AbsoluteScale_up', 'JES_AbsoluteScale_down']
perform inference
Inference and selection for jshift 
Inference and selection for jshift JER_up
Inference and selection for jshift JER_down
Inference and selection for jshift JES_AbsoluteScale_up
Inference and selection for jshift JES_AbsoluteScale_down
load samples hh4b
Loading GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV
Loaded GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV: 48963 entries
scaling and smearing mass for hh4b 2022
JEC shifts  ['', 'JER_up', 'JER_down', 'JES_AbsoluteScale_up', 'JES_AbsoluteScale_down', 'JMS_up', 'JMS_down', 'JMR_up', 'JMR_down']
perform inference
Inference and selection for jshift 
Inference and selection for jshift JER_up
Inference an

/home/users/dprimosc/HH4b/src/HH4b/utils.py:232: UserWarning: Temporarily scaling VBFHHto4B_CV_1p74_C2V_1p37_C3_14p4_TuneCP5_13p6TeV_madgraph-pythia8 by its xsec and lumi - remember to remove after fixing in the processor!
  warnings.warn(


Loaded VBFHHto4B_CV_1p74_C2V_1p37_C3_14p4_TuneCP5_13p6TeV_madgraph-pythia8: 118910 entries
scaling and smearing mass for vbfhh4b-kv1p74-k2v1p37-kl14p4 2023
JEC shifts  ['', 'JER_up', 'JER_down', 'JES_AbsoluteScale_up', 'JES_AbsoluteScale_down', 'JMS_up', 'JMS_down', 'JMR_up', 'JMR_down']
perform inference
Inference and selection for jshift 
Inference and selection for jshift JER_up
Inference and selection for jshift JER_down
Inference and selection for jshift JES_AbsoluteScale_up
Inference and selection for jshift JES_AbsoluteScale_down
Inference and selection for jshift JMS_up
Inference and selection for jshift JMS_down
Inference and selection for jshift JMR_up
Inference and selection for jshift JMR_down
load samples vbfhh4b-kvm0p962-k2v0p959-klm1p43
Loading VBFHHto4B_CV_m0p962_C2V_0p959_C3_m1p43_TuneCP5_13p6TeV_madgraph-pythia8
Loaded VBFHHto4B_CV_m0p962_C2V_0p959_C3_m1p43_TuneCP5_13p6TeV_madgraph-pythia8: 1428 entries
scaling and smearing mass for vbfhh4b-kvm0p962-k2v0p959-klm1p43 2

...

/home/users/dprimosc/HH4b/src/HH4b/utils.py:232: UserWarning: Temporarily scaling VBFHHto4B_CV_m1p83_C2V_3p57_C3_m3p39_TuneCP5_13p6TeV_madgraph-pythia8 by its xsec and lumi - remember to remove after fixing in the processor!
  warnings.warn(


Loaded VBFHHto4B_CV_m1p83_C2V_3p57_C3_m3p39_TuneCP5_13p6TeV_madgraph-pythia8: 72029 entries
scaling and smearing mass for vbfhh4b-kvm1p83-k2v3p57-klm3p39 2023BPix
JEC shifts  ['', 'JER_up', 'JER_down', 'JES_AbsoluteScale_up', 'JES_AbsoluteScale_down', 'JMS_up', 'JMS_down', 'JMR_up', 'JMR_down']
perform inference
Inference and selection for jshift 
Inference and selection for jshift JER_up
Inference and selection for jshift JER_down
Inference and selection for jshift JES_AbsoluteScale_up
Inference and selection for jshift JES_AbsoluteScale_down
Inference and selection for jshift JMS_up
Inference and selection for jshift JMS_down
Inference and selection for jshift JMR_up
Inference and selection for jshift JMR_down
load samples vbfhh4b-kvm2p12-k2v3p87-klm5p96
Loading VBFHHto4B_CV_m2p12_C2V_3p87_C3_m5p96_TuneCP5_13p6TeV_madgraph-pythia8


/home/users/dprimosc/HH4b/src/HH4b/utils.py:232: UserWarning: Temporarily scaling VBFHHto4B_CV_m2p12_C2V_3p87_C3_m5p96_TuneCP5_13p6TeV_madgraph-pythia8 by its xsec and lumi - remember to remove after fixing in the processor!
  warnings.warn(


Loaded VBFHHto4B_CV_m2p12_C2V_3p87_C3_m5p96_TuneCP5_13p6TeV_madgraph-pythia8: 44596 entries
scaling and smearing mass for vbfhh4b-kvm2p12-k2v3p87-klm5p96 2023BPix
JEC shifts  ['', 'JER_up', 'JER_down', 'JES_AbsoluteScale_up', 'JES_AbsoluteScale_down', 'JMS_up', 'JMS_down', 'JMR_up', 'JMR_down']
perform inference
Inference and selection for jshift 
Inference and selection for jshift JER_up
Inference and selection for jshift JER_down
Inference and selection for jshift JES_AbsoluteScale_up
Inference and selection for jshift JES_AbsoluteScale_down
Inference and selection for jshift JMS_up
Inference and selection for jshift JMS_down
Inference and selection for jshift JMR_up
Inference and selection for jshift JMR_down

Cutflow
                                   Skimmer Preselection          HLT  \
data                                        546004.0000  521166.0000   
ttbar                                        39716.6943   30932.8759   
gghtobb                                        143.31

'\npython3 PostProcess.py --templates-tag 24June27 --tag 24May24_v12_private_signal --mass H2PNetMass --legacy --bdt-config 24May31_lr_0p02_md_8_AK4Away --bdt-model 24May31_lr_0p02_md_8_AK4Away --txbb-wps 0.975 0.92 --bdt-wps 0.98 0.88 0.03 --vbf-txbb-wp 0.95 --vbf-bdt-wp 0.98 --no-bdt-roc --no-fom-scan --no-fom-scan-bin2 --no-fom-scan-bin1 --data-dir /ceph/cms/store/user/cmantill/bbbb/skimmer/ --method abcd --no-vbf-priority --vbf --no-fom-scan-vbf --pt-second 250 --templates --years 2022 --sig-keys hh4b vbfhh4b\n'

In [26]:
import uproot
import pandas as pd


eventlist_dict = ["event", "bdt_score", "bdt_score_vbf", "H2TXbb", "H2Msd", "run", "H2PNetMass"]

# Loop over all years and save event lists for each year in separate root files
for year, ev_dict in ev_dicts:

    hh4b_df = ev_dict["hh4b"]
    event_list = hh4b_df[eventlist_dict]

    array_to_save = {col: event_list[col].to_numpy() for col in event_list.columns}

    with uproot.recreate(f"eventlist_boostedHH4b_{year}.root") as file:
        file["tree"] = array_to_save

In [28]:
# Double-check file contents

dfs_from_root = {}

for year in years:
    filename = f"eventlist_boostedHH4b_{year}.root"
    with uproot.open(filename) as file:
        tree = file["tree"]
        arrays = tree.arrays(library="np")
        df = pd.DataFrame(arrays)

        # Make dict
        dfs_from_root[year] = df

# Display df
for year, df in dfs_from_root.items():
    print(f"DataFrame for year {year}:\n{df}\n")

DataFrame for year 2022:
         event  bdt_score  bdt_score_vbf    H2TXbb      H2Msd  run  H2PNetMass
0      1450699   0.777012       0.648450  0.000226  127.18750    1  126.742600
1      1450708   0.829893       0.179832  0.985870  119.00000    1  143.878378
2      1503251   0.860970       0.192123  0.662430   46.03125    1   83.081488
3      1503311   0.609028       0.843063  0.547774  104.56250    1  104.581630
4      1503323   0.069489       0.003767  0.794878    7.68750    1   90.861472
...        ...        ...            ...       ...        ...  ...         ...
38743  1644555   0.756790       0.281042  0.956689  118.87500    1  108.395992
38744  1644579   0.974344       0.938412  0.993592  146.75000    1  134.922604
38745  1644584   0.965913       0.652996  0.977377  138.50000    1  141.811446
38746  2011738   0.877202       0.202041  0.964582   97.75000    1  110.795429
38747  2011784   0.937218       0.469333  0.981580  165.12500    1  175.498111

[38748 rows x 7 columns]

